In [19]:
from skimage import io as skimageio
import imageio
from skimage.transform import rescale, resize, downscale_local_mean, resize_local_mean
import zarr
import numpy as np
import napari
from skimage.filters import rank 
from skimage.morphology import ball
from scipy.ndimage import maximum_filter, minimum_filter, rank_filter
from skimage.filters import gaussian
from virtualRG import utils
from skimage.measure import regionprops, regionprops_table

In [19]:
from skimage import io as skimageio
import imageio
from skimage.transform import rescale, resize, downscale_local_mean, resize_local_mean
import zarr
import numpy as np
import napari
from skimage.filters import rank 
from skimage.morphology import ball
from scipy.ndimage import maximum_filter, minimum_filter, rank_filter
from skimage.filters import gaussian
from virtualRG import utils
from skimage.measure import regionprops, regionprops_table

In [2]:
# prepare .npy file for TubeMap.
zarr_path = '/mnt/ampa_data01/tmurakami/220305_SMA_nuc_middlehuman/ch488_10um.zarr'
nuclei_zarr = zarr.open(zarr_path, mode='r')
# load images as zarr
nuclei = nuclei_zarr['0'][:]
voxel_size = [10, 10, 10]


In [33]:
%%time
vertical_footprint = np.ones((5,10,1))
per_high = rank_filter(nuclei,rank=5,footprint=vertical_footprint)#maximum_filter(nuclei,footprint=vertical_footprint)
per_low = rank_filter(nuclei,rank=-5,footprint=vertical_footprint)#minimum_filter(nuclei,footprint=vertical_footprint)

gap_cond = (per_low - per_high)>100

nuclei_norm = ((nuclei - per_high) / (per_low - per_high)) * gap_cond
nuclei_norm[np.isnan(nuclei_norm)] = 0

#autofl_gauss = gaussian(nuclei_norm,sigma=10,mode='nearest',preserve_range=True)

CPU times: user 2min 15s, sys: 210 ms, total: 2min 15s
Wall time: 2min 15s


In [54]:
viewer = napari.Viewer()
#viewer.add_image(autofl_gauss, contrast_limits=[0,50], rgb=False, name='filter', colormap='magenta', blending='additive', multiscale=False)
viewer.add_image(nuclei, contrast_limits=[0,2000], rgb=False, name='nuc', colormap='green', blending='additive', multiscale=False)
viewer.add_image(tes, contrast_limits=[0,2000], rgb=False, name='nuc', colormap='green', blending='additive', multiscale=False)

<Image layer 'nuc [1]' at 0x7f8d55d26ee0>

In [24]:
# use dask to analyze segmented nuclei
import dask_image
from dask_image import ndmeasure
import dask.array as da
import dask
import pandas as pd

In [68]:
# from dask.distributed import Client, LocalCluster
# cluster = LocalCluster(dashboard_address='localhost:8787')
# client = Client(cluster) # https://docs.dask.org/en/latest/how-to/deploy-dask/single-distributed.html
# client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 64,Total memory: 503.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39445,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 503.52 GiB
Comm: tcp://127.0.0.1:38653,Total threads: 8
Dashboard: http://127.0.0.1:45361/status,Memory: 62.94 GiB
Nanny: tcp://127.0.0.1:43927,


In [4]:
segmented_nuc = da.from_zarr('/mnt/ampa_data01/tmurakami/220305_SMA_nuc_middlehuman/fin_segmentation.zarr')# max to be 17925870

In [67]:
### This failed.
# idx = np.arange(17925870)[1:]
# ones_arr = da.ones_like(segmented_nuc)
# areas = ndmeasure.area(ones_arr, label_image=segmented_nuc, index=idx)
# centers = ndmeasure.center_of_mass(ones_arr, label_image=segmented_nuc, index=idx)
# areas_ = areas.compute()
# centers_ = centers.compute()

In [26]:
import functools
import operator

def get_block_iter(image):
    block_iter = zip(
        np.ndindex(*image.numblocks),
        map(
            functools.partial(operator.getitem, image),
            da.core.slices_from_chunks(image.chunks),
        ),
    )
    return block_iter

In [8]:
segmented_nuc

,Array,Chunk
Bytes,589.39 GiB,256.00 MiB
Shape,"(1489, 12567, 8455)","(256, 512, 512)"
Count,2551 Tasks,2550 Chunks
Type,int32,numpy.ndarray


In [31]:
depth = (40,40,40)
boundary=(0,0,0)
props_blocks = np.empty(segmented_nuc.numblocks, dtype=object)
segmented_nuc_overlap = da.overlap.overlap(segmented_nuc, depth, boundary)
block_iter = get_block_iter(segmented_nuc_overlap)
for index, input_block in block_iter:
    #props = dask.delayed(regionprops_table)(input_block,properties=('label', 'area', 'centroid', 'bbox'))
    props = regionprops_table(input_block.compute(),properties=('label', 'area', 'centroid', 'bbox'))
    props_blocks[index]=props

In [82]:
%%time
df = pd.DataFrame()
chunk_info = segmented_nuc.chunks
res = np.asarray(depth)

for index in np.ndindex(*segmented_nuc.numblocks):
    corner_coord = np.asarray([sum(chunk_info[i][:index[i]]) for i in range(3)])
    # adjust the coordinates to be consistent with the global coordinates.
    regional_df = pd.DataFrame(props_blocks[index])
    regional_df[['centroid-0','centroid-1','centroid-2']] = regional_df[['centroid-0','centroid-1','centroid-2']]-res+corner_coord
    regional_df[['bbox-0','bbox-1','bbox-2']] = regional_df[['bbox-0','bbox-1','bbox-2']]-res+corner_coord
    regional_df[['bbox-3','bbox-4','bbox-5']] = regional_df[['bbox-3','bbox-4','bbox-5']]-res+corner_coord
    
    # append to the table
    df = pd.concat([df,regional_df],axis=0)
# save to csv if necessary
# df.to_csv('/mnt/ampa_data01/tmurakami/220305_SMA_nuc_middlehuman/coord_duplicated.csv')

CPU times: user 2min 8s, sys: 2min 33s, total: 4min 42s
Wall time: 4min 43s


In [101]:
df = df.sort_values(by=['label'])

# get only duplicated cells 
dup = df['label'].duplicated(keep=False)
df_dup = df[dup].reset_index(drop=True)

# get the cells with largest area within the same label
idx = df_dup.groupby(['label'])['area'].transform(max) == df_dup['area']
df_dup = df_dup[idx]

# it is possible that the cells with the same label shows the same max areas. remove them.
df_dup = df_dup[~df_dup['label'].duplicated()]
df_unique = pd.concat([df[~dup],df_dup],axis=0)

# sort and clean up the dataframe
df_unique = df_unique.sort_values(by=['label']).reset_index(drop=True)

# save to csv if necessary
# df_unique.to_csv('/mnt/ampa_data01/tmurakami/220305_SMA_nuc_middlehuman/coord_centroids.csv')

,label,area,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5
0,1516,93,147.086022,509.817204,2011.827957,144,508,2009,151,512,2016
4,1518,834,151.059952,493.117506,2044.925659,145,488,2040,158,499,2052
8,1519,849,154.135453,508.343934,2002.062426,146,500,1995,164,512,2009
12,1520,1134,153.734568,478.432099,2039.886243,148,472,2032,161,486,2048
16,1521,144,151.458333,496.680556,2020.312500,148,493,2017,156,500,2024
...,...,...,...,...,...,...,...,...,...,...,...
10808777,17925866,195,1482.830769,3548.861538,2225.641026,1482,3543,2222,1485,3557,2231
10808779,17925867,104,1482.644231,3555.423077,2061.826923,1482,3550,2059,1485,3561,2066
10808783,17925868,122,1482.622951,3574.704918,2141.655738,1482,3570,2138,1485,3582,2147
10808785,17925869,145,1482.510345,3576.834483,2240.813793,1482,3573,2236,1484,3583,2247


In [174]:
# https://napari.org/api/napari.view_layers.html#napari.view_layers.view_points
viewer = napari.view_points(
    coords,
    size=20,
    edge_width=0,
    symbol='disc',
    out_of_slice_display=True
)

/home/tmurakami/app/miniconda3/envs/deform/lib/python3.8/site-packages/numpy/core/numeric.py:2448: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [173]:
#infos = df_unique.sample(n=1000000,random_state=1)[['area','centroid-0','centroid-1','centroid-2']]
infos = df_unique[['area','centroid-0','centroid-1','centroid-2']]
coords = infos[['centroid-0','centroid-1','centroid-2']].to_numpy() * np.asarray([3,0.65,0.65]) # scaling 
coords = coords.astype(int)
size = infos[['area']].to_numpy().flatten()
constant = 5.
size = np.power(size,2/3) / constant 

In [168]:
size

array([109.00042084,  72.24019575,  96.63825298, ...,  70.90054182,
        32.93375139, 130.86229533])